In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Load your data
data = {
    'sex': [1.0, 1.0, 0.0, 1.0, 0.0],
    'age': [48, 46, 60, 41, 48],
    'education': [2, 2, 3, 1, 2],
    'smokingStatus': [0.0, 1.0, 0.0, 1.0, 1.0],
    'cigsPerDay': [0, 20, 0, 4, 40],
    'BPMeds': [0, 0, 0, 0, 0],
    'prevalentStroke': [0, 0, 0, 0, 0],
    'prevalentHyp': [0, 1, 1, 0, 0],
    'diabetes': [0, 0, 0, 0, 0],
    'totChol': [237, 271, 276, 176, 226],
    'sysBP': [124.5, 158.0, 144.0, 113.0, 117.5],
    'diaBP': [66.5, 94.0, 78.0, 75.0, 80.0],
    'BMI': [33.29, 25.17, 26.98, 22.29, 26.18],
    'heartRate': [80, 78, 60, 80, 60],
    'glucose': [91, 71, 88, 55, 66],
    'CHDRisk': [0, 0, 0, 0, 0]
}

df = pd.DataFrame(data)

# Splitting the dataset
features = df.drop('CHDRisk', axis=1).values
targets = df['CHDRisk'].values
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# Create datasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=10, shuffle=False)




In [2]:
# Define the neural network architecture
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(X_train.shape[1], 64)
        self.layer2 = nn.Linear(64, 64)
        self.output_layer = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = torch.sigmoid(self.output_layer(x))
        return x

# Instantiate the model
model = NeuralNetwork()

In [3]:

# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')

# Evaluate the model
model.eval()
test_loss = 0
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)
        test_loss += loss.item()

        predicted = (outputs.squeeze() > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_loss /= len(test_loader)
accuracy = 100 * correct / total
print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%')


Epoch [1/50], Loss: 0.7245, Accuracy: 25.00%
Epoch [2/50], Loss: 0.7056, Accuracy: 25.00%
Epoch [3/50], Loss: 0.6873, Accuracy: 50.00%
Epoch [4/50], Loss: 0.6694, Accuracy: 100.00%
Epoch [5/50], Loss: 0.6522, Accuracy: 100.00%
Epoch [6/50], Loss: 0.6353, Accuracy: 100.00%
Epoch [7/50], Loss: 0.6184, Accuracy: 100.00%
Epoch [8/50], Loss: 0.6017, Accuracy: 100.00%
Epoch [9/50], Loss: 0.5851, Accuracy: 100.00%
Epoch [10/50], Loss: 0.5691, Accuracy: 100.00%
Epoch [11/50], Loss: 0.5534, Accuracy: 100.00%
Epoch [12/50], Loss: 0.5383, Accuracy: 100.00%
Epoch [13/50], Loss: 0.5233, Accuracy: 100.00%
Epoch [14/50], Loss: 0.5083, Accuracy: 100.00%
Epoch [15/50], Loss: 0.4930, Accuracy: 100.00%
Epoch [16/50], Loss: 0.4774, Accuracy: 100.00%
Epoch [17/50], Loss: 0.4615, Accuracy: 100.00%
Epoch [18/50], Loss: 0.4455, Accuracy: 100.00%
Epoch [19/50], Loss: 0.4292, Accuracy: 100.00%
Epoch [20/50], Loss: 0.4127, Accuracy: 100.00%
Epoch [21/50], Loss: 0.3961, Accuracy: 100.00%
Epoch [22/50], Loss: 0.37

ValueError: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.